# Bệnh dịch Tân sinh viên

*Mô hình hóa và mô phỏng bằng Python*

Bản quyền 2021 Allen Downey

Giấy phép: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [16]:
# cài đặt Pint nếu cần

try:
    import pint
except ImportError:
    !pip install pint

In [17]:
# tải về modsim.py nếu cần

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/modsim.py')

In [18]:
# import functions from modsim

from modsim import *

In [24]:
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'chap11.py')

In [25]:
# nhập mã lệnh từ các sổ notebooks trước

from chap11 import make_system
from chap11 import update_func
from chap11 import run_simulation

[Nhấn vào đây để chạy nghiên cứu cụ thể này trên Colab](https://colab.research.google.com/github/AllenDowney/ModSimPy/blob/master/examples/plague.ipynb)

Nghiên cứu cụ thể này tiếp tục từ điểm dừng ở Chương 12.

In [26]:
def add_immunization(system, fraction):
    system.init.S -= fraction
    system.init.R += fraction

In [27]:
tc = 3             # thời gian giữa các lượt tiếp xúc, đơn vị ngày 
tr = 4             # thời gian hồi phục, đơn vị ngày

beta = 1 / tc      # tốc độ tiếp xúc, đơn vị 1/ngày
gamma = 1 / tr     # tốc độ hồi phục, đơn vị 1/ngày

system = make_system(beta, gamma)

In [28]:
def calc_total_infected(results, system):
    s_0 = results.S[0]
    s_end = results.S[system.t_end]
    return s_0 - s_end

## Rửa tay

Giả sử bạn là Trưởng ban Đời sống sinh viên, và bạn chỉ có quỹ \$1200  để chống lại Bệnh dịch tân sinh viên. Bạn có hai phương án tiêu số tiền này:

1.  Bạn có thể chi trả việc tiêm vắc-xin với giá \$100 mỗi liều.

2.  Bạn có thể tiêu tiền để vận động tuyên truyền cho sinh viên rửa tay
    thường xuyên.

Ta đã thấy rằng có thể mô hình hóa hiệu ứng của vắc-xin như thế nào rồi. 
Giờ ta hãy nghĩ về vận động tuyên truyền rửa tay. Ta sẽ phải trả lời 
hai câu hỏi:

1.  Làm thế nào để đưa hiệu ứng của việc rửa tay vào mô hình này?

2.  Làm thế nào để định lượng hiệu ứng của số tiền dành cho tổ chức vận động
    tuyên truyền cho rửa tay?

Để đơn giản, hãy giả sử rằng ta đã có dữ liệu của một đợt vận động tương tự
ở một trường học khác, trong đó cho thấy rằng một đợt vận động được đầu tư đầy đủ
sẽ làm thay đổi hành động của cộng đồng sinh viên đến mức giảm được tỉ lệ nhiễm bớt được 20%.

In terms of the model, hand washing has the effect of reducing `beta`.
That's not the only way we could incorporate the effect, but it seems
reasonable and it's easy to implement.

Now we have to model the relationship between the money we spend and the
effectiveness of the campaign. Again, let's suppose we have data from
another school that suggests:

-   If we spend \$500 on posters, materials, and staff time, we can
    change student behavior in a way that decreases the effective value of `beta` by 10%.

-   If we spend \$1000, the total decrease in `beta` is almost 20%.

-   Above \$1000, additional spending has little additional benefit.

## Logistic function

To model the effect of a hand-washing campaign, I'll use a [generalized logistic function](https://en.wikipedia.org/wiki/Generalised_logistic_function) (GLF), which is a convenient function for modeling curves that have a generally sigmoid shape.  The parameters of the GLF correspond to various features of the curve in a way that makes it easy to find a function that has the shape you want, based on data or background information about the scenario.

In [29]:
from numpy import exp

def logistic(x, A=0, B=1, C=1, M=0, K=1, Q=1, nu=1):
    """Tính hàm logit tổng quát.
    
    A: kiểm soát cận dưới
    B: kiểm soát độ dốc chuyển giao 
    C: theo tôi thì tham số này vô dụng
    M: kiểm soát vị trí của chuyển giao
    K: kiểm soát cận dưới
    Q: dịch chuyển giao qua trái hoặc phải
    nu: ảnh hưởng tính đối xứng của chuyển giao
    
    trả lại: float hoặc array
    """
    exponent = -B * (x - M)
    denom = C + Q * exp(exponent)
    return A + (K-A) / denom ** (1/nu)

Mảng sau đây thể hiện khoảng các mức chi khả dĩ.

In [30]:
spending = linspace(0, 1200, 21)

`compute_factor` tính mức giảm `beta` với một khoản chi vận động cho trước.

`M` được chọn sao cho chuyển giao nằm ở quanh mức \$500.

`K` là mức giảm tối đa của `beta`, 20%.

`B` được chọn qua cách thử sai nhằm thu được một đường cong khả dĩ.

In [31]:
def compute_factor(spending):
    """Hệ số chiết giảm như một hàm phụ thuộc vào mức chi.
    
    spending: mức chi tính theo đôla từ 0 đến 1200
    
    trả lại: tỉ lệ phần giảm của beta
    """
    return logistic(spending, M=500, K=0.2, B=0.01)

Here's what it looks like.

In [32]:
percent_reduction = compute_factor(spending) * 100

make_series(spending, percent_reduction).plot()

decorate(xlabel='Hand-washing campaign spending (USD)',
         ylabel='Percent reduction in infection rate',
         title='Effect of hand washing on infection rate')

The result is the following function, which
takes spending as a parameter and returns `factor`, which is the factor
by which `beta` is reduced:

In [33]:
def compute_factor(spending):
    return logistic(spending, M=500, K=0.2, B=0.01)

I use `compute_factor` to write `add_hand_washing`, which takes a
`System` object and a budget, and modifies `system.beta` to model the
effect of hand washing:

In [34]:
def add_hand_washing(system, spending):
    factor = compute_factor(spending)
    system.beta *= (1 - factor)

Now we can sweep a range of values for `spending` and use the simulation
to compute the effect:

In [35]:
def sweep_hand_washing(spending_array):
    sweep = SweepSeries()
    
    for spending in spending_array:
        system = make_system(beta, gamma)
        add_hand_washing(system, spending)
        results = run_simulation(system, update_func)
        sweep[spending] = calc_total_infected(results, system)
        
    return sweep

Here's how we run it:

In [36]:
from numpy import linspace

spending_array = linspace(0, 1200, 20)
infected_sweep2 = sweep_hand_washing(spending_array)

The following figure shows the result. 

In [37]:
infected_sweep2.plot()

decorate(xlabel='Hand-washing campaign spending (USD)',
         ylabel='Total fraction infected',
         title='Effect of hand washing on total infections')

Below \$200, the campaign has little effect. 

At \$800 it has a substantial effect, reducing total infections from more than 45% to about 20%. 

Above \$800, the additional benefit is small.

## Optimization

Let's put it all together. With a fixed budget of \$1200, we have to
decide how many doses of vaccine to buy and how much to spend on the
hand-washing campaign.

Here are the parameters:

In [38]:
num_students = 90
budget = 1200
price_per_dose = 100
max_doses = int(budget / price_per_dose)
max_doses

The fraction `budget/price_per_dose` might not be an integer. `int` is a
built-in function that converts numbers to integers, rounding down.

We'll sweep the range of possible doses:

In [39]:
dose_array = linrange(max_doses)

In this example we call `linrange` with only one argument; it returns a NumPy array with the integers from 0 to `max_doses`, including both.

Then we run the simulation for each element of `dose_array`:

In [40]:
def sweep_doses(dose_array):
    sweep = SweepSeries()
    
    for doses in dose_array:
        fraction = doses / num_students
        spending = budget - doses * price_per_dose
        
        system = make_system(beta, gamma)
        add_immunization(system, fraction)
        add_hand_washing(system, spending)
        
        results = run_simulation(system, update_func)
        sweep[doses] = calc_total_infected(results, system)

    return sweep

For each number of doses, we compute the fraction of students we can
immunize, `fraction` and the remaining budget we can spend on the
campaign, `spending`. Then we run the simulation with those quantities
and store the number of infections.

The following figure shows the result.

In [41]:
infected_sweep3 = sweep_doses(dose_array)

In [42]:
infected_sweep3.plot()

decorate(xlabel='Doses of vaccine',
         ylabel='Total fraction infected',
         title='Total infections vs. doses')

If we buy no doses of vaccine and spend the entire budget on the campaign, the fraction infected is around 19%. At 4 doses, we have \$800 left for the campaign, and this is the optimal point that minimizes the number of students who get sick.

As we increase the number of doses, we have to cut campaign spending,
which turns out to make things worse. But interestingly, when we get
above 10 doses, the effect of herd immunity starts to kick in, and the
number of sick students goes down again.

**Exercise:** Suppose the price of the vaccine drops to $50 per dose.  How does that affect the optimal allocation of the spending?